In [7]:
import pandas as pd
import random
import os
from faker import Faker

# Configuração
fake = Faker('pt_BR') # Gera nomes em Português
random.seed(42)
Faker.seed(42)

num_clientes = 2999

# Bairros Estratégicos
bairros_alvorada = ['B', 'C', 'D', 'F', 'G', 'H', 'I', 'J']
bairros_poa = ['A', 'E', 'I', 'O', 'U']

lista_clientes = []


In [8]:
print("--- Gerando Clientes com ID_Data ---")

for i in range(num_clientes):
    # Clientes de Porto Alegre
    if random.random() < 0.40:
        cidade = 'Porto Alegre'
        bairro = random.choice(bairros_alvorada) # Nota: No código original você inverteu as listas de bairro aqui, mantive a lógica original
    else:
        cidade = 'Alvorada'
        bairro = random.choice(bairros_poa)
    
    # Perfil demográfico
    genero = random.choice(['M', 'F'])
    nome = fake.name_male() if genero == 'M' else fake.name_female()
    
    # Segmentação
    perfil = random.choices(['Premium', 'Caçador de Oferta', 'Padrão'], weights=[10, 30, 60])[0]
    
    # --- CORREÇÃO AQUI: GERAR DATA COMO INTEIRO (YYYYMMDD) ---
    data_random = fake.date_between(start_date='-5y', end_date='today')
    id_data_cadastro = int(data_random.strftime('%Y%m%d')) # Transforma em Inteiro: Ex: 20231225
    
    lista_clientes.append({
        'id_cliente': i + 1000,
        'nome': nome,
        'genero': genero,
        'idade': random.randint(18, 85),
        'cidade': cidade,
        'bairro': bairro,
        'perfil_compra': perfil,
        'id_data_cadastro': id_data_cadastro # Agora compatível com dim_data
    })



--- Gerando Clientes com ID_Data ---


In [9]:
# Criar e Padronizar DataFrame
df_clientes = pd.DataFrame(lista_clientes)

def padronizar_dados_para_bd(df_input):
    """
    Função para limpar metadados e padronizar o DataFrame para inserção em Banco de Dados.
    
    Ações:
    1. Remove espaços extras de colunas de texto (strip).
    2. Padroniza nomes das colunas para Snake Case (minusculo_com_underline).
    3. Garante que colunas numéricas tenham a tipagem correta.
    4. Remove duplicatas acidentais.
    """
    # Cria uma cópia para não alterar o original sem querer
    df = df_input.copy()
    
    # 1. Limpeza de Strings (remove espaços vazios no início/fim)
    # Itera apenas sobre colunas do tipo 'object' (texto)
    colunas_texto = df.select_dtypes(include=['object']).columns
    for col in colunas_texto:
       # Pega o primeiro valor válido (não nulo) para testar o tipo
        primeiro_valor_valido_idx = df[col].first_valid_index()
        
        if primeiro_valor_valido_idx is not None:
            valor_amostra = df[col].loc[primeiro_valor_valido_idx]
            
            # Só aplica o strip se a amostra for realmente TEXTO (str)
            if isinstance(valor_amostra, str):
                df[col] = df[col].str.strip()
    
    # 2. Padronização de Colunas (Snake Case)
    df.columns = (df.columns
                  .str.lower()            # Tudo para minúsculo
                  .str.replace(' ', '_')  # Espaços viram underline
                  .str.replace('/', '_')  # Barras viram underline
                  .str.normalize('NFKD')  # Remove acentuação
                  .str.encode('ascii', errors='ignore')
                  .str.decode('utf-8')
    )
    
    # 3. Garantia de Tipagem Numérica
    colunas_numericas = df.select_dtypes(include=['number']).columns
    for col in colunas_numericas:
        if not col.startswith('id_') and not col.endswith('_id') and col != 'id':
            df[col] = df[col].astype(float).round(2)
            
    # 4. Remove duplicatas (se houver linhas idênticas)
    df = df.drop_duplicates()
    
    return df

# Aplique a função no seu DataFrame original
df_clientes_limpo = padronizar_dados_para_bd(df_clientes)

# Visualizar a diferença nos nomes das colunas
print("Colunas antigas:", df_clientes.columns.tolist())
print("Colunas novas:  ", df_clientes_limpo.columns.tolist())

# Atualize a variável principal para salvar o CSV correto
df_clientes = df_clientes_limpo


Colunas antigas: ['id_cliente', 'nome', 'genero', 'idade', 'cidade', 'bairro', 'perfil_compra', 'id_data_cadastro']
Colunas novas:   ['id_cliente', 'nome', 'genero', 'idade', 'cidade', 'bairro', 'perfil_compra', 'id_data_cadastro']


In [10]:
# Visualizar os primeiros itens
print("Amostra da Tabela de Produtos Gerada:")
print(df_clientes.head(10))

Amostra da Tabela de Produtos Gerada:
   id_cliente                    nome genero  idade        cidade bairro  \
0        1000      Srta. Amanda Sousa      F   35.0      Alvorada      A   
1        1001  Henry Gabriel Carvalho      M   22.0      Alvorada      U   
2        1002      Sr. Otávio Andrade      M   21.0  Porto Alegre      F   
3        1003     Helena da Conceição      F   53.0      Alvorada      U   
4        1004           Apollo Novais      M   61.0      Alvorada      A   
5        1005          Dra. Ísis Leão      F   66.0  Porto Alegre      F   
6        1006              Lara Abreu      F   76.0  Porto Alegre      H   
7        1007       Dr. Luigi da Mota      M   64.0      Alvorada      O   
8        1008          Lucca Caldeira      M   55.0      Alvorada      A   
9        1009           Calebe Fogaça      M   76.0      Alvorada      E   

       perfil_compra  id_data_cadastro  
0  Caçador de Oferta          20230426  
1             Padrão          20211106  
2 

In [11]:
# Salvar em CSV
caminho_pasta = r'D:\Projetos\MC_Supermercados' 

# Verifica se a pasta existe, se não, salva na pasta atual mesmo e avisa
if os.path.exists(caminho_pasta):
    caminho_completo = os.path.join(caminho_pasta, 'dim_clientes.csv')
    df_clientes.to_csv(caminho_completo, index=False, encoding='utf-8-sig')
    print(f"Arquivo salvo com sucesso em: {caminho_completo}\n")
else:
    # Se você não definiu o caminho acima, ele salva na pasta atual e te diz onde é
    df_clientes.to_csv('dim_clientes.csv', index=False, encoding='utf-8-sig')
    print("Pasta específica não encontrada ou não definida.")
    print(f"O arquivo foi salvo na pasta padrão do Python: {os.getcwd()}")
    print("Nome do arquivo: dim_produtos.csv")

Arquivo salvo com sucesso em: D:\Projetos\MC_Supermercados\dim_clientes.csv

